In [1]:
import pandas as pd
from datetime import datetime as dt
import numpy as np

In [2]:
# read in dataset 
ce6 = pd.read_csv('../data/slurm_wrapper_ce6.log',
                  header=None,
                  delimiter=' - ',
                  engine='python',
                #   nrows=10000
                  )

In [3]:
# set column names
colnames = ['END', 'USER', 'RETRY', 'TIME', 'RETURNCODE', 'COMMAND']
ce6.columns = colnames

from carb_funcs import optional_ms

# Apply the parsing function to the 'END' column
ce6['END'] = optional_ms(ce6['END'])

# Clean and convert other columns
ce6['TIME'] = ce6['TIME'].str.replace('time', '', regex=False).astype(float)
ce6['USER'] = ce6['USER'].str.replace('user', '', regex=False).astype(int)
ce6['RETRY'] = ce6['RETRY'].str.replace('retry', '', regex=False).astype(int)
ce6['RETURNCODE'] = ce6['RETURNCODE'].str.replace('returncode', '', regex=False).astype(float)
ce6['COMMAND'] = ce6['COMMAND'].str.replace('command', '', regex=False)

# Display the updated DataFrame
ce6.head(2)

,END,USER,RETRY,TIME,RETURNCODE,COMMAND
0,2020-10-16 10:37:44.163454,9202,0,0.084954,0.0,"['/usr/bin/scontrol', 'show', 'job', '24997187']"
1,2020-10-16 10:37:44.206654,9202,0,0.089431,0.0,"['/usr/bin/scontrol', 'show', 'job', '24997190']"


In [4]:
# checking that all values contain timestamps
print(ce6['END'].sort_values().iloc[-1])

2021-10-07 22:00:20.919558


In [ ]:
# slicing out the records that satisfy unresponsive conditions outlined. 
ce6_unresponsive_df = ce6[(ce6['USER'] == 9204) & (ce6['RETURNCODE'] == 1.0) & (ce6['TIME'] > 15)]
display(ce6_unresponsive_df.head(3))
print(ce6_unresponsive_df.info(), ce6_unresponsive_df.shape)

,END,USER,RETRY,TIME,RETURNCODE,COMMAND
10335,2020-10-16 22:16:06.507040,9204,0,20.018827,1.0,"['/usr/bin/scontrol', 'show', 'job']"
10337,2020-10-16 22:16:26.548379,9204,1,20.040872,1.0,"['/usr/bin/scontrol', 'show', 'job']"
10943,2020-10-16 22:31:09.071034,9204,0,20.033828,1.0,"['/usr/bin/scontrol', 'show', 'job']"


<class 'pandas.core.frame.DataFrame'>
Index: 6472 entries, 10335 to 4766868
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   END         6472 non-null   datetime64[ns]
 1   USER        6472 non-null   int64         
 2   RETRY       6472 non-null   int64         
 3   TIME        6472 non-null   float64       
 4   RETURNCODE  6472 non-null   float64       
 5   COMMAND     6472 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(1)
memory usage: 353.9+ KB
None (6472, 6)
